# Preload precipitation files

Since loading from the original COSMO GRIB files takes quite a long time, I will try to preload them in a separate file. This should hopefully speed up my analysis.

In [1]:
# Imports
import helpers; reload(helpers)
from helpers import *
from config import *
import os
import numpy as np
import datetime as dt
from cosmo_utils.helpers import ddhhmmss, make_timelist, yyyymmddhhmmss_strtotime, yyyymmddhhmmss
import timeit

ATTENTION: You are using a very early, definitely not bug-free netCDF version of getfobj. Please report bugs to s.rasp@lmu.de or fix them ;)


In [2]:
exp_ids = ['DA_REF', 'DA_PSPv2', 'DA_REF_TL500', 'DA_PSPv2_TL500', 'noDA_PSPv2']
date_start = '20160526000000'
date_stop = '20160608120000'
hint = 12
fclt = 24   # Forecast lead time

In [3]:
date_list = make_timelist(
    yyyymmddhhmmss_strtotime(date_start),
    yyyymmddhhmmss_strtotime(date_stop),
    dt.timedelta(hours=hint),
)

In [4]:
date_list[:5]

[datetime.datetime(2016, 5, 26, 0, 0),
 datetime.datetime(2016, 5, 26, 12, 0),
 datetime.datetime(2016, 5, 27, 0, 0),
 datetime.datetime(2016, 5, 27, 12, 0),
 datetime.datetime(2016, 5, 28, 0, 0)]

## Preload deterministic forecasts

In [5]:
def preload_det_prec_for_one_day(exp_id, date_str, fclt=24):
    data_dir = datadir + exp_id + '/'
    for h in range(1, fclt +1):
        hstr = ddhhmmss(dt.timedelta(hours=h))
        try:
            p = load_det(data_dir, date_str, hstr, return_array=True)
        except Exception:
            print 'File missing:', data_dir, date_str, hstr
            p = np.nan
        if h == 1:  # Allocate output array
            prec = np.ones((fclt, p.shape[0], p.shape[1])) * np.nan
        prec[h - 1] = p
    # Save
    save_fn = savedir_base + exp_id + '/prec_fields/' 
    if not os.path.exists(save_fn):
        os.makedirs(save_fn)
    save_fn += 'det_' +  date_str + '.npy'
    np.save(save_fn, prec)

In [41]:
preload_det_prec_for_one_day('DA_REF', date_start)

initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160526000000/det/lfff00010000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160526000000/det/lfff00000000_15
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160526000000/det/lfff00020000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160526000000/det/lfff00010000_15
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160526000000/det/lfff00030000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160526000000/det/lfff00020000_15
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160526000000/det/lfff00040000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160526000000/det/lfff00030000_15
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160526

## Preload ensemble forecasts

In [34]:
def preload_ens_prec_for_one_day(exp_id, date_str, fclt=24):
    data_dir = datadir + exp_id + '_ens/'
    for h in range(1, fclt +1):
        hstr = ddhhmmss(dt.timedelta(hours=h))
        try:
            p = np.array(load_ens(data_dir, date_str, hstr, return_array=True))
        except Exception:
            print 'File missing:', data_dir, date_str, hstr
            p = np.nan
        if h == 1:  # Allocate output array
            prec = np.ones((fclt, p.shape[0], p.shape[1], p.shape[2])) * np.nan
        prec[h - 1] = p
    # Save
    save_fn = savedir_base + exp_id + '/prec_fields/' 
    if not os.path.exists(save_fn):
        os.makedirs(save_fn)
    save_fn += 'ens_' +  date_str + '.npy'
    np.save(save_fn, prec)

In [47]:
preload_ens_prec_for_one_day('DA_REF', date_start)

read to tmp-file /tmp/user/13098/tmp9okG27fieldobj_list_array_met-ws-720e07_10156.npy
reading took 0:00:03.646483
read to tmp-file /tmp/user/13098/tmp9okG27fieldobj_list_array_met-ws-720e07_10156.npy
reading took 0:00:04.738707
read to tmp-file /tmp/user/13098/tmp9okG27fieldobj_list_array_met-ws-720e07_10156.npy
reading took 0:00:06.395695
read to tmp-file /tmp/user/13098/tmp9okG27fieldobj_list_array_met-ws-720e07_10156.npy
reading took 0:00:04.334186
read to tmp-file /tmp/user/13098/tmp9okG27fieldobj_list_array_met-ws-720e07_10156.npy
reading took 0:00:04.423728
read to tmp-file /tmp/user/13098/tmp9okG27fieldobj_list_array_met-ws-720e07_10156.npy
reading took 0:00:04.442428
read to tmp-file /tmp/user/13098/tmp9okG27fieldobj_list_array_met-ws-720e07_10156.npy
reading took 0:00:04.723308
read to tmp-file /tmp/user/13098/tmp9okG27fieldobj_list_array_met-ws-720e07_10156.npy
reading took 0:00:04.542957
read to tmp-file /tmp/user/13098/tmp9okG27fieldobj_list_array_met-ws-720e07_10156.npy
re

## Loop over experiments and dates

In [48]:
for exp_id in exp_ids:
    print exp_id
    for date in date_list:
        print date
        date_str = yyyymmddhhmmss(date)
        preload_det_prec_for_one_day(exp_id, date_str)
        preload_ens_prec_for_one_day(exp_id, date_str)

DA_REF
2016-05-26 00:00:00
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160526000000/det/lfff00010000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160526000000/det/lfff00000000_15
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160526000000/det/lfff00020000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160526000000/det/lfff00010000_15
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160526000000/det/lfff00030000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160526000000/det/lfff00020000_15
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160526000000/det/lfff00040000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160526000000/det/lfff00030000_15
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/da

reading took 0:00:03.567355
2016-05-26 12:00:00
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160526120000/det/lfff00010000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160526120000/det/lfff00000000_15
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160526120000/det/lfff00020000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160526120000/det/lfff00010000_15
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160526120000/det/lfff00030000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160526120000/det/lfff00020000_15
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160526120000/det/lfff00040000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160526120000/det/lfff00030000_15
initial path:  /project/meteo/w2w/A6/S.R

reading took 0:00:04.030199
2016-05-27 00:00:00
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160527000000/det/lfff00010000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160527000000/det/lfff00000000_15
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160527000000/det/lfff00020000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160527000000/det/lfff00010000_15
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160527000000/det/lfff00030000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160527000000/det/lfff00020000_15
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160527000000/det/lfff00040000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160527000000/det/lfff00030000_15
initial path:  /project/meteo/w2w/A6/S.R

reading took 0:00:03.875127
2016-05-27 12:00:00
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160527120000/det/lfff00010000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160527120000/det/lfff00000000_15
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160527120000/det/lfff00020000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160527120000/det/lfff00010000_15
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160527120000/det/lfff00030000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160527120000/det/lfff00020000_15
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160527120000/det/lfff00040000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160527120000/det/lfff00030000_15
initial path:  /project/meteo/w2w/A6/S.R

reading took 0:00:04.095630
2016-05-28 00:00:00
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160528000000/det/lfff00010000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160528000000/det/lfff00000000_15
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160528000000/det/lfff00020000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160528000000/det/lfff00010000_15
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160528000000/det/lfff00030000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160528000000/det/lfff00020000_15
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160528000000/det/lfff00040000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160528000000/det/lfff00030000_15
initial path:  /project/meteo/w2w/A6/S.R

...

initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_PSPv2_TL500/20160608000000/det/lfff00030000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_PSPv2_TL500/20160608000000/det/lfff00020000_15
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_PSPv2_TL500/20160608000000/det/lfff00040000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_PSPv2_TL500/20160608000000/det/lfff00030000_15
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_PSPv2_TL500/20160608000000/det/lfff00050000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_PSPv2_TL500/20160608000000/det/lfff00040000_15
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_PSPv2_TL500/20160608000000/det/lfff00060000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_PSPv2_TL500/20160608000000/det/lfff00050000_15
initial path:  /project/

initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_PSPv2_TL500/20160608120000/det/lfff00020000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_PSPv2_TL500/20160608120000/det/lfff00010000_15
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_PSPv2_TL500/20160608120000/det/lfff00030000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_PSPv2_TL500/20160608120000/det/lfff00020000_15
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_PSPv2_TL500/20160608120000/det/lfff00040000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_PSPv2_TL500/20160608120000/det/lfff00030000_15
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_PSPv2_TL500/20160608120000/det/lfff00050000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_PSPv2_TL500/20160608120000/det/lfff00040000_15
initial path:  /project/

reading took 0:00:04.047074
read to tmp-file /tmp/user/13098/tmp9okG27fieldobj_list_array_met-ws-720e07_10156.npy
reading took 0:00:04.162770
noDA_PSPv2
2016-05-26 00:00:00
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/noDA_PSPv2/20160526000000/det/lfff00010000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/noDA_PSPv2/20160526000000/det/lfff00000000_15
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/noDA_PSPv2/20160526000000/det/lfff00020000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/noDA_PSPv2/20160526000000/det/lfff00010000_15
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/noDA_PSPv2/20160526000000/det/lfff00030000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/noDA_PSPv2/20160526000000/det/lfff00020000_15
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/noDA_PSPv2/20160526000000/det/lfff00040000_15


reading took 0:00:04.344571
read to tmp-file /tmp/user/13098/tmp9okG27fieldobj_list_array_met-ws-720e07_10156.npy
reading took 0:00:04.216893
read to tmp-file /tmp/user/13098/tmp9okG27fieldobj_list_array_met-ws-720e07_10156.npy
reading took 0:00:04.593591
read to tmp-file /tmp/user/13098/tmp9okG27fieldobj_list_array_met-ws-720e07_10156.npy
reading took 0:00:04.153214
2016-05-26 12:00:00
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/noDA_PSPv2/20160526120000/det/lfff00010000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/noDA_PSPv2/20160526120000/det/lfff00000000_15
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/noDA_PSPv2/20160526120000/det/lfff00020000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/noDA_PSPv2/20160526120000/det/lfff00010000_15
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/noDA_PSPv2/20160526120000/det/lfff00030000_15
1h earlier:  /proje

Exception: Requested member file does not exist:/project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/noDA_PSPv2_ens//20160526120000/ens016/lfff00120000_15

Some data is missing for the noDA runs. I built in an exception for these cases in the loading functions where missing files are filled with NaNs.

In [35]:
for date in date_list:
    print date
    date_str = yyyymmddhhmmss(date)
    # preload_det_prec_for_one_day('noDA_PSPv2', date_str)
    preload_ens_prec_for_one_day('noDA_PSPv2', date_str)

2016-05-26 00:00:00
read to tmp-file /tmp/user/13098/tmpYFoKZKfieldobj_list_array_met-ws-720e07_28141.npy
reading took 0:01:13.911158
read to tmp-file /tmp/user/13098/tmpYFoKZKfieldobj_list_array_met-ws-720e07_28141.npy
reading took 0:01:02.236207
read to tmp-file /tmp/user/13098/tmpYFoKZKfieldobj_list_array_met-ws-720e07_28141.npy
reading took 0:01:03.271657
read to tmp-file /tmp/user/13098/tmpYFoKZKfieldobj_list_array_met-ws-720e07_28141.npy
reading took 0:00:59.557993
read to tmp-file /tmp/user/13098/tmpYFoKZKfieldobj_list_array_met-ws-720e07_28141.npy
reading took 0:00:58.649855
read to tmp-file /tmp/user/13098/tmpYFoKZKfieldobj_list_array_met-ws-720e07_28141.npy
reading took 0:01:04.582895
read to tmp-file /tmp/user/13098/tmpYFoKZKfieldobj_list_array_met-ws-720e07_28141.npy
reading took 0:01:04.632550
read to tmp-file /tmp/user/13098/tmpYFoKZKfieldobj_list_array_met-ws-720e07_28141.npy
reading took 0:01:10.955256
read to tmp-file /tmp/user/13098/tmpYFoKZKfieldobj_list_array_met-ws

reading took 0:00:08.292067
2016-05-27 12:00:00
read to tmp-file /tmp/user/13098/tmpYFoKZKfieldobj_list_array_met-ws-720e07_28141.npy
reading took 0:00:09.308964
read to tmp-file /tmp/user/13098/tmpYFoKZKfieldobj_list_array_met-ws-720e07_28141.npy
reading took 0:00:07.463241
read to tmp-file /tmp/user/13098/tmpYFoKZKfieldobj_list_array_met-ws-720e07_28141.npy
reading took 0:00:10.892490
read to tmp-file /tmp/user/13098/tmpYFoKZKfieldobj_list_array_met-ws-720e07_28141.npy
reading took 0:00:08.458891
read to tmp-file /tmp/user/13098/tmpYFoKZKfieldobj_list_array_met-ws-720e07_28141.npy
reading took 0:00:09.195655
read to tmp-file /tmp/user/13098/tmpYFoKZKfieldobj_list_array_met-ws-720e07_28141.npy
reading took 0:00:08.759373
read to tmp-file /tmp/user/13098/tmpYFoKZKfieldobj_list_array_met-ws-720e07_28141.npy
reading took 0:00:09.214068
read to tmp-file /tmp/user/13098/tmpYFoKZKfieldobj_list_array_met-ws-720e07_28141.npy
reading took 0:00:13.007896
read to tmp-file /tmp/user/13098/tmpYFoK

reading took 0:00:04.896159
2016-05-29 00:00:00
read to tmp-file /tmp/user/13098/tmpYFoKZKfieldobj_list_array_met-ws-720e07_28141.npy
reading took 0:00:07.759568
read to tmp-file /tmp/user/13098/tmpYFoKZKfieldobj_list_array_met-ws-720e07_28141.npy
reading took 0:00:05.096038
read to tmp-file /tmp/user/13098/tmpYFoKZKfieldobj_list_array_met-ws-720e07_28141.npy
reading took 0:00:05.722349
read to tmp-file /tmp/user/13098/tmpYFoKZKfieldobj_list_array_met-ws-720e07_28141.npy
reading took 0:00:05.466602
read to tmp-file /tmp/user/13098/tmpYFoKZKfieldobj_list_array_met-ws-720e07_28141.npy
reading took 0:00:08.543023
read to tmp-file /tmp/user/13098/tmpYFoKZKfieldobj_list_array_met-ws-720e07_28141.npy
reading took 0:00:06.574763
read to tmp-file /tmp/user/13098/tmpYFoKZKfieldobj_list_array_met-ws-720e07_28141.npy
reading took 0:00:06.072363
read to tmp-file /tmp/user/13098/tmpYFoKZKfieldobj_list_array_met-ws-720e07_28141.npy
reading took 0:00:05.877764
read to tmp-file /tmp/user/13098/tmpYFoK

reading took 0:00:06.002488
2016-05-30 12:00:00
read to tmp-file /tmp/user/13098/tmpYFoKZKfieldobj_list_array_met-ws-720e07_28141.npy
reading took 0:00:06.222371
read to tmp-file /tmp/user/13098/tmpYFoKZKfieldobj_list_array_met-ws-720e07_28141.npy
reading took 0:00:05.826720
read to tmp-file /tmp/user/13098/tmpYFoKZKfieldobj_list_array_met-ws-720e07_28141.npy
reading took 0:00:05.748114
read to tmp-file /tmp/user/13098/tmpYFoKZKfieldobj_list_array_met-ws-720e07_28141.npy
reading took 0:00:05.351639
read to tmp-file /tmp/user/13098/tmpYFoKZKfieldobj_list_array_met-ws-720e07_28141.npy
reading took 0:00:06.257371
read to tmp-file /tmp/user/13098/tmpYFoKZKfieldobj_list_array_met-ws-720e07_28141.npy
reading took 0:00:06.206494
read to tmp-file /tmp/user/13098/tmpYFoKZKfieldobj_list_array_met-ws-720e07_28141.npy
reading took 0:00:05.947052
read to tmp-file /tmp/user/13098/tmpYFoKZKfieldobj_list_array_met-ws-720e07_28141.npy
reading took 0:00:06.300811
read to tmp-file /tmp/user/13098/tmpYFoK

reading took 0:00:04.705584
2016-06-01 00:00:00
read to tmp-file /tmp/user/13098/tmpYFoKZKfieldobj_list_array_met-ws-720e07_28141.npy
reading took 0:00:05.973616
read to tmp-file /tmp/user/13098/tmpYFoKZKfieldobj_list_array_met-ws-720e07_28141.npy
reading took 0:00:04.718630
read to tmp-file /tmp/user/13098/tmpYFoKZKfieldobj_list_array_met-ws-720e07_28141.npy
reading took 0:00:04.599816
read to tmp-file /tmp/user/13098/tmpYFoKZKfieldobj_list_array_met-ws-720e07_28141.npy
reading took 0:00:05.023462
read to tmp-file /tmp/user/13098/tmpYFoKZKfieldobj_list_array_met-ws-720e07_28141.npy
reading took 0:00:04.908070
read to tmp-file /tmp/user/13098/tmpYFoKZKfieldobj_list_array_met-ws-720e07_28141.npy
reading took 0:00:05.846972
read to tmp-file /tmp/user/13098/tmpYFoKZKfieldobj_list_array_met-ws-720e07_28141.npy
reading took 0:00:05.790927
read to tmp-file /tmp/user/13098/tmpYFoKZKfieldobj_list_array_met-ws-720e07_28141.npy
reading took 0:00:05.544937
read to tmp-file /tmp/user/13098/tmpYFoK

reading took 0:00:04.631909
2016-06-02 12:00:00
read to tmp-file /tmp/user/13098/tmpYFoKZKfieldobj_list_array_met-ws-720e07_28141.npy
reading took 0:00:05.502418
read to tmp-file /tmp/user/13098/tmpYFoKZKfieldobj_list_array_met-ws-720e07_28141.npy
reading took 0:00:04.775323
read to tmp-file /tmp/user/13098/tmpYFoKZKfieldobj_list_array_met-ws-720e07_28141.npy
reading took 0:00:05.123614
read to tmp-file /tmp/user/13098/tmpYFoKZKfieldobj_list_array_met-ws-720e07_28141.npy
reading took 0:00:05.308651
read to tmp-file /tmp/user/13098/tmpYFoKZKfieldobj_list_array_met-ws-720e07_28141.npy
reading took 0:00:04.989118
read to tmp-file /tmp/user/13098/tmpYFoKZKfieldobj_list_array_met-ws-720e07_28141.npy
reading took 0:00:04.818627
read to tmp-file /tmp/user/13098/tmpYFoKZKfieldobj_list_array_met-ws-720e07_28141.npy
reading took 0:00:04.633071
read to tmp-file /tmp/user/13098/tmpYFoKZKfieldobj_list_array_met-ws-720e07_28141.npy
reading took 0:00:04.929394
read to tmp-file /tmp/user/13098/tmpYFoK

reading took 0:00:05.805586
2016-06-04 00:00:00
read to tmp-file /tmp/user/13098/tmpYFoKZKfieldobj_list_array_met-ws-720e07_28141.npy
reading took 0:00:07.159877
read to tmp-file /tmp/user/13098/tmpYFoKZKfieldobj_list_array_met-ws-720e07_28141.npy
reading took 0:00:06.797529
read to tmp-file /tmp/user/13098/tmpYFoKZKfieldobj_list_array_met-ws-720e07_28141.npy
reading took 0:00:05.025556
read to tmp-file /tmp/user/13098/tmpYFoKZKfieldobj_list_array_met-ws-720e07_28141.npy
reading took 0:00:04.932025
read to tmp-file /tmp/user/13098/tmpYFoKZKfieldobj_list_array_met-ws-720e07_28141.npy
reading took 0:00:04.872361
read to tmp-file /tmp/user/13098/tmpYFoKZKfieldobj_list_array_met-ws-720e07_28141.npy
reading took 0:00:05.767793
read to tmp-file /tmp/user/13098/tmpYFoKZKfieldobj_list_array_met-ws-720e07_28141.npy
reading took 0:00:06.652693
read to tmp-file /tmp/user/13098/tmpYFoKZKfieldobj_list_array_met-ws-720e07_28141.npy
reading took 0:00:05.657247
read to tmp-file /tmp/user/13098/tmpYFoK

reading took 0:00:04.749034
2016-06-05 12:00:00
read to tmp-file /tmp/user/13098/tmpYFoKZKfieldobj_list_array_met-ws-720e07_28141.npy
reading took 0:00:05.445809
read to tmp-file /tmp/user/13098/tmpYFoKZKfieldobj_list_array_met-ws-720e07_28141.npy
reading took 0:00:05.836590
read to tmp-file /tmp/user/13098/tmpYFoKZKfieldobj_list_array_met-ws-720e07_28141.npy
reading took 0:00:05.704997
read to tmp-file /tmp/user/13098/tmpYFoKZKfieldobj_list_array_met-ws-720e07_28141.npy
reading took 0:00:04.788725
read to tmp-file /tmp/user/13098/tmpYFoKZKfieldobj_list_array_met-ws-720e07_28141.npy
reading took 0:00:06.260407
read to tmp-file /tmp/user/13098/tmpYFoKZKfieldobj_list_array_met-ws-720e07_28141.npy
reading took 0:00:06.622123
read to tmp-file /tmp/user/13098/tmpYFoKZKfieldobj_list_array_met-ws-720e07_28141.npy
reading took 0:00:06.350536
read to tmp-file /tmp/user/13098/tmpYFoKZKfieldobj_list_array_met-ws-720e07_28141.npy
reading took 0:00:04.841436
read to tmp-file /tmp/user/13098/tmpYFoK

reading took 0:00:04.824711
2016-06-07 00:00:00
read to tmp-file /tmp/user/13098/tmpYFoKZKfieldobj_list_array_met-ws-720e07_28141.npy
reading took 0:00:06.963390
read to tmp-file /tmp/user/13098/tmpYFoKZKfieldobj_list_array_met-ws-720e07_28141.npy
reading took 0:00:04.793361
read to tmp-file /tmp/user/13098/tmpYFoKZKfieldobj_list_array_met-ws-720e07_28141.npy
reading took 0:00:04.720671
read to tmp-file /tmp/user/13098/tmpYFoKZKfieldobj_list_array_met-ws-720e07_28141.npy
reading took 0:00:05.612150
read to tmp-file /tmp/user/13098/tmpYFoKZKfieldobj_list_array_met-ws-720e07_28141.npy
reading took 0:00:05.734702
read to tmp-file /tmp/user/13098/tmpYFoKZKfieldobj_list_array_met-ws-720e07_28141.npy
reading took 0:00:05.998458
read to tmp-file /tmp/user/13098/tmpYFoKZKfieldobj_list_array_met-ws-720e07_28141.npy
reading took 0:00:05.908194
read to tmp-file /tmp/user/13098/tmpYFoKZKfieldobj_list_array_met-ws-720e07_28141.npy
reading took 0:00:05.583504
read to tmp-file /tmp/user/13098/tmpYFoK

reading took 0:00:04.649510
2016-06-08 12:00:00
read to tmp-file /tmp/user/13098/tmpYFoKZKfieldobj_list_array_met-ws-720e07_28141.npy
reading took 0:00:05.500566
read to tmp-file /tmp/user/13098/tmpYFoKZKfieldobj_list_array_met-ws-720e07_28141.npy
reading took 0:00:04.339279
read to tmp-file /tmp/user/13098/tmpYFoKZKfieldobj_list_array_met-ws-720e07_28141.npy
reading took 0:00:04.848714
read to tmp-file /tmp/user/13098/tmpYFoKZKfieldobj_list_array_met-ws-720e07_28141.npy
reading took 0:00:04.918128
read to tmp-file /tmp/user/13098/tmpYFoKZKfieldobj_list_array_met-ws-720e07_28141.npy
reading took 0:00:04.316873
read to tmp-file /tmp/user/13098/tmpYFoKZKfieldobj_list_array_met-ws-720e07_28141.npy
reading took 0:00:05.110516
read to tmp-file /tmp/user/13098/tmpYFoKZKfieldobj_list_array_met-ws-720e07_28141.npy
reading took 0:00:04.953171
read to tmp-file /tmp/user/13098/tmpYFoKZKfieldobj_list_array_met-ws-720e07_28141.npy
reading took 0:00:05.049144
read to tmp-file /tmp/user/13098/tmpYFoK

## Preload radar observations

In [13]:
def preload_radar_prec_for_one_day(date_str, fclt=24):
    for h in range(1, fclt +1):
        hstr = ddhhmmss(dt.timedelta(hours=h))
        p = load_radar(date_str, hstr, return_array=True);
        if h == 1:  # Allocate output array
            prec = np.empty((fclt, p.shape[0], p.shape[1]))
        prec[h - 1] = p
    # Save
    save_fn = savedir_base + 'radar/prec_fields/' 
    if not os.path.exists(save_fn):
        os.makedirs(save_fn)
    save_fn += 'radar_' +  date_str + '.npy'
    np.save(save_fn, prec)

In [14]:
preload_radar_prec_for_one_day(date_start)

20160526000000 00010000 1
20160526000000 00020000 2
20160526000000 00030000 3
20160526000000 00040000 4
20160526000000 00050000 5
20160526000000 00060000 6
20160526000000 00070000 7
20160526000000 00080000 8
20160526000000 00090000 9
20160526000000 00100000 10
20160526000000 00110000 11
20160526000000 00120000 12
20160526000000 00130000 13
20160526000000 00140000 14
20160526000000 00150000 15
20160526000000 00160000 16
20160526000000 00170000 17
20160526000000 00180000 18
20160526000000 00190000 19
20160526000000 00200000 20
20160526000000 00210000 21
20160526000000 00220000 22
20160526000000 00230000 23
20160526000000 01000000 24


In [17]:
for date in date_list:
    print date
    date_str = yyyymmddhhmmss(date)
    preload_radar_prec_for_one_day(date_str)

2016-05-26 00:00:00
2016-05-26 12:00:00
2016-05-27 00:00:00
2016-05-27 12:00:00
2016-05-28 00:00:00
2016-05-28 12:00:00
2016-05-29 00:00:00
2016-05-29 12:00:00
2016-05-30 00:00:00
2016-05-30 12:00:00
2016-05-31 00:00:00
2016-05-31 12:00:00
2016-06-01 00:00:00
2016-06-01 12:00:00
2016-06-02 00:00:00
2016-06-02 12:00:00
2016-06-03 00:00:00
2016-06-03 12:00:00
2016-06-04 00:00:00
2016-06-04 12:00:00
2016-06-05 00:00:00
2016-06-05 12:00:00
2016-06-06 00:00:00
2016-06-06 12:00:00
2016-06-07 00:00:00
2016-06-07 12:00:00
2016-06-08 00:00:00
2016-06-08 12:00:00


## Preload CAPE and CIN

In [19]:
def preload_det_cape_cin_for_one_day(exp_id, date_str, fclt=24):
    data_dir = datadir + exp_id + '/'
    for h in range(1, fclt +1):
        hstr = ddhhmmss(dt.timedelta(hours=h))
        ca, ci = load_det_cape_cin(data_dir, date_str, hstr, return_array=True)
        ci[ci < 0] = np.nan
        if h == 1:  # Allocate output array
            cape = np.ones((fclt, ca.shape[0], ca.shape[1])) * np.nan
            cin = np.ones((fclt, ca.shape[0], ca.shape[1])) * np.nan
        cape[h - 1] = ca
        cin[h - 1] = ci
    # Save
    save_fn_cape = savedir_base + exp_id + '/cape_fields/' 
    if not os.path.exists(save_fn_cape):
        os.makedirs(save_fn_cape)
    save_fn_cape += 'det_' +  date_str + '.npy'
    np.save(save_fn_cape, cape)
    save_fn_cin = savedir_base + exp_id + '/cin_fields/' 
    if not os.path.exists(save_fn_cin):
        os.makedirs(save_fn_cin)
    save_fn_cin += 'det_' +  date_str + '.npy'
    np.save(save_fn_cin, cin)

In [20]:
for exp_id in exp_ids:
    print exp_id
    for date in date_list:
        print date
        date_str = yyyymmddhhmmss(date)
        preload_det_cape_cin_for_one_day(exp_id, date_str)

DA_REF
2016-05-26 00:00:00
2016-05-26 12:00:00
2016-05-27 00:00:00
2016-05-27 12:00:00
2016-05-28 00:00:00
2016-05-28 12:00:00
2016-05-29 00:00:00
2016-05-29 12:00:00
2016-05-30 00:00:00
2016-05-30 12:00:00
2016-05-31 00:00:00
2016-05-31 12:00:00
2016-06-01 00:00:00
2016-06-01 12:00:00
2016-06-02 00:00:00
2016-06-02 12:00:00
2016-06-03 00:00:00
2016-06-03 12:00:00
2016-06-04 00:00:00
2016-06-04 12:00:00
2016-06-05 00:00:00
2016-06-05 12:00:00
2016-06-06 00:00:00
2016-06-06 12:00:00
2016-06-07 00:00:00
2016-06-07 12:00:00
2016-06-08 00:00:00
2016-06-08 12:00:00
DA_PSPv2
2016-05-26 00:00:00
2016-05-26 12:00:00
2016-05-27 00:00:00
2016-05-27 12:00:00
2016-05-28 00:00:00
2016-05-28 12:00:00
2016-05-29 00:00:00
2016-05-29 12:00:00
2016-05-30 00:00:00
2016-05-30 12:00:00
2016-05-31 00:00:00
2016-05-31 12:00:00
2016-06-01 00:00:00
2016-06-01 12:00:00
2016-06-02 00:00:00
2016-06-02 12:00:00
2016-06-03 00:00:00
2016-06-03 12:00:00
2016-06-04 00:00:00
2016-06-04 12:00:00
2016-06-05 00:00:00
2016

## Preload deterministic first guess

How are the filest structured:
- Directories are 3 hourly (eg 20160526090000)
- They contain lff_prec.* files from -1 to +1h, which contain the accumulation from 0 to +1h
- but laf files from 0 to +2h

So which is the corresponding radar observation?
- The radar observations for a given hour contain the accumulation from -1 to 0h

So the corresponding radar observation is +1h from the first guess forecast. 

For daily saves the first time step is 0 to 1UTC, which would be saved in the 0UTC file. The last time step is 23 to 24UTC which would be in the 23UTC file


In [5]:
d_shape = (461, 421)

In [15]:
def preload_fg_det_prec_for_one_day(exp_id, date_obj, fclt=24):
    # Get 3hrly storage dir
    date_str = date_str = yyyymmddhhmmss(date_obj)
    for h in range(1, fclt+1):
        td = dt.timedelta(hours=h)
        t = date_obj + td
        date_ana = yyyymmddhhmmss(t)
        date_fg = yyyymmddhhmmss(t - timedelta(hours=1))
        date_store = yyyymmddhhmmss(t - timedelta(hours=t.hour % 3))
        
        ana_dir = (datadir_da + exp_id + '/' + date_store + '/')
        hstr = ddhhmmss(dt.timedelta(hours=h))
        try:
            p = load_det_da(ana_dir, date_fg, return_array=True)
        except Exception:
            print 'File missing:', ana_dir, date_fg
            p = np.nan
        if h == 1:  # Allocate output array
            prec = np.ones((fclt, d_shape[0], d_shape[1])) * np.nan
        prec[h - 1] = p
    # Save
    save_fn = savedir_base + exp_id + '/prec_fields/' 
    if not os.path.exists(save_fn):
        os.makedirs(save_fn)
    save_fn += 'det_da_' +  date_str + '.npy'
    np.save(save_fn, prec)

## Preload ensemble first guess

In [16]:
n_ens_da = 40

In [17]:
def preload_fg_ens_prec_for_one_day(exp_id, date_obj, fclt=24):
    # Get 3hrly storage dir
    date_str = date_str = yyyymmddhhmmss(date_obj)
    for h in range(1, fclt + 1):
        td = dt.timedelta(hours=h)
        t = date_obj + td
        date_ana = yyyymmddhhmmss(t)
        date_fg = yyyymmddhhmmss(t - timedelta(hours=1))
        date_store = yyyymmddhhmmss(t - timedelta(hours=t.hour % 3))
        
        ana_dir = (datadir_da + exp_id + '/' + date_store + '/')
        hstr = ddhhmmss(dt.timedelta(hours=h))
        try:
            p = np.array(load_ens_da(ana_dir, date_fg, return_array=True))
        except Exception:
            print 'File missing:', ana_dir, date_fg
            p = np.nan
        if h == 1:  # Allocate output array
            prec = np.ones((fclt, n_ens_da, d_shape[0], d_shape[1])) * np.nan
        prec[h - 1] = p
    # Save
    save_fn = savedir_base + exp_id + '/prec_fields/' 
    if not os.path.exists(save_fn):
        os.makedirs(save_fn)
    save_fn += 'ens_da_' +  date_str + '.npy'
    np.save(save_fn, prec)

In [18]:
for exp_id in exp_ids[:-1]:
    print exp_id
    for date in date_list[1:]:
        print date
        preload_fg_det_prec_for_one_day(exp_id, date)
        preload_fg_ens_prec_for_one_day(exp_id, date)

DA_REF
2016-05-26 12:00:00
read to tmp-file /tmp/user/13098/tmpkE0Y7wfieldobj_list_array_met-ws-720e07_6316.npy
reading took 0:00:04.985693
read to tmp-file /tmp/user/13098/tmpkE0Y7wfieldobj_list_array_met-ws-720e07_6316.npy
reading took 0:00:04.372433
read to tmp-file /tmp/user/13098/tmpkE0Y7wfieldobj_list_array_met-ws-720e07_6316.npy
reading took 0:00:04.645651
read to tmp-file /tmp/user/13098/tmpkE0Y7wfieldobj_list_array_met-ws-720e07_6316.npy
reading took 0:00:04.591714
read to tmp-file /tmp/user/13098/tmpkE0Y7wfieldobj_list_array_met-ws-720e07_6316.npy
reading took 0:00:04.874699
read to tmp-file /tmp/user/13098/tmpkE0Y7wfieldobj_list_array_met-ws-720e07_6316.npy
reading took 0:00:04.522383
read to tmp-file /tmp/user/13098/tmpkE0Y7wfieldobj_list_array_met-ws-720e07_6316.npy
reading took 0:00:04.380319
read to tmp-file /tmp/user/13098/tmpkE0Y7wfieldobj_list_array_met-ws-720e07_6316.npy
reading took 0:00:04.582863
read to tmp-file /tmp/user/13098/tmpkE0Y7wfieldobj_list_array_met-ws-

2016-05-28 00:00:00
read to tmp-file /tmp/user/13098/tmpkE0Y7wfieldobj_list_array_met-ws-720e07_6316.npy
reading took 0:00:04.214899
read to tmp-file /tmp/user/13098/tmpkE0Y7wfieldobj_list_array_met-ws-720e07_6316.npy
reading took 0:00:04.155882
read to tmp-file /tmp/user/13098/tmpkE0Y7wfieldobj_list_array_met-ws-720e07_6316.npy
reading took 0:00:03.954381
read to tmp-file /tmp/user/13098/tmpkE0Y7wfieldobj_list_array_met-ws-720e07_6316.npy
reading took 0:00:04.452733
read to tmp-file /tmp/user/13098/tmpkE0Y7wfieldobj_list_array_met-ws-720e07_6316.npy
reading took 0:00:03.760420
read to tmp-file /tmp/user/13098/tmpkE0Y7wfieldobj_list_array_met-ws-720e07_6316.npy
reading took 0:00:04.000727
read to tmp-file /tmp/user/13098/tmpkE0Y7wfieldobj_list_array_met-ws-720e07_6316.npy
reading took 0:00:03.990775
read to tmp-file /tmp/user/13098/tmpkE0Y7wfieldobj_list_array_met-ws-720e07_6316.npy
reading took 0:00:04.498399
read to tmp-file /tmp/user/13098/tmpkE0Y7wfieldobj_list_array_met-ws-720e07_

2016-05-29 12:00:00
read to tmp-file /tmp/user/13098/tmpkE0Y7wfieldobj_list_array_met-ws-720e07_6316.npy
reading took 0:00:03.932789
read to tmp-file /tmp/user/13098/tmpkE0Y7wfieldobj_list_array_met-ws-720e07_6316.npy
reading took 0:00:03.651099
read to tmp-file /tmp/user/13098/tmpkE0Y7wfieldobj_list_array_met-ws-720e07_6316.npy
reading took 0:00:03.740372
read to tmp-file /tmp/user/13098/tmpkE0Y7wfieldobj_list_array_met-ws-720e07_6316.npy
reading took 0:00:03.648577
read to tmp-file /tmp/user/13098/tmpkE0Y7wfieldobj_list_array_met-ws-720e07_6316.npy
reading took 0:00:03.760409
read to tmp-file /tmp/user/13098/tmpkE0Y7wfieldobj_list_array_met-ws-720e07_6316.npy
reading took 0:00:03.923483
read to tmp-file /tmp/user/13098/tmpkE0Y7wfieldobj_list_array_met-ws-720e07_6316.npy
reading took 0:00:03.771581
read to tmp-file /tmp/user/13098/tmpkE0Y7wfieldobj_list_array_met-ws-720e07_6316.npy
reading took 0:00:03.826861
read to tmp-file /tmp/user/13098/tmpkE0Y7wfieldobj_list_array_met-ws-720e07_

2016-05-31 00:00:00
read to tmp-file /tmp/user/13098/tmpkE0Y7wfieldobj_list_array_met-ws-720e07_6316.npy
reading took 0:00:04.296531
read to tmp-file /tmp/user/13098/tmpkE0Y7wfieldobj_list_array_met-ws-720e07_6316.npy
reading took 0:00:04.084285
read to tmp-file /tmp/user/13098/tmpkE0Y7wfieldobj_list_array_met-ws-720e07_6316.npy
reading took 0:00:05.348162
read to tmp-file /tmp/user/13098/tmpkE0Y7wfieldobj_list_array_met-ws-720e07_6316.npy
reading took 0:00:04.843474
read to tmp-file /tmp/user/13098/tmpkE0Y7wfieldobj_list_array_met-ws-720e07_6316.npy
reading took 0:00:05.160886
read to tmp-file /tmp/user/13098/tmpkE0Y7wfieldobj_list_array_met-ws-720e07_6316.npy
reading took 0:00:05.924594
read to tmp-file /tmp/user/13098/tmpkE0Y7wfieldobj_list_array_met-ws-720e07_6316.npy
reading took 0:00:04.135544
read to tmp-file /tmp/user/13098/tmpkE0Y7wfieldobj_list_array_met-ws-720e07_6316.npy
reading took 0:00:03.976433
read to tmp-file /tmp/user/13098/tmpkE0Y7wfieldobj_list_array_met-ws-720e07_

2016-06-01 12:00:00
File missing: /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data/DA_REF/20160602000000/ 20160601230000
read to tmp-file /tmp/user/13098/tmpkE0Y7wfieldobj_list_array_met-ws-720e07_6316.npy
reading took 0:00:04.485046
read to tmp-file /tmp/user/13098/tmpkE0Y7wfieldobj_list_array_met-ws-720e07_6316.npy
reading took 0:00:03.704101
read to tmp-file /tmp/user/13098/tmpkE0Y7wfieldobj_list_array_met-ws-720e07_6316.npy
reading took 0:00:04.906446
read to tmp-file /tmp/user/13098/tmpkE0Y7wfieldobj_list_array_met-ws-720e07_6316.npy
reading took 0:00:04.543570
read to tmp-file /tmp/user/13098/tmpkE0Y7wfieldobj_list_array_met-ws-720e07_6316.npy
reading took 0:00:04.052725
read to tmp-file /tmp/user/13098/tmpkE0Y7wfieldobj_list_array_met-ws-720e07_6316.npy
reading took 0:00:03.721976
read to tmp-file /tmp/user/13098/tmpkE0Y7wfieldobj_list_array_met-ws-720e07_6316.npy
reading took 0:00:04.353157
read to tmp-file /tmp/user/13098/tmpkE0Y7wfieldobj_list_array_met-ws-720e07_6316.npy
rea

...

2016-06-01 12:00:00
read to tmp-file /tmp/user/13098/tmpkE0Y7wfieldobj_list_array_met-ws-720e07_6316.npy
reading took 0:00:04.124324
read to tmp-file /tmp/user/13098/tmpkE0Y7wfieldobj_list_array_met-ws-720e07_6316.npy
reading took 0:00:04.021654
read to tmp-file /tmp/user/13098/tmpkE0Y7wfieldobj_list_array_met-ws-720e07_6316.npy
reading took 0:00:03.903656
read to tmp-file /tmp/user/13098/tmpkE0Y7wfieldobj_list_array_met-ws-720e07_6316.npy
reading took 0:00:03.889812
read to tmp-file /tmp/user/13098/tmpkE0Y7wfieldobj_list_array_met-ws-720e07_6316.npy
reading took 0:00:03.927684
read to tmp-file /tmp/user/13098/tmpkE0Y7wfieldobj_list_array_met-ws-720e07_6316.npy
reading took 0:00:04.124365
read to tmp-file /tmp/user/13098/tmpkE0Y7wfieldobj_list_array_met-ws-720e07_6316.npy
reading took 0:00:03.882388
read to tmp-file /tmp/user/13098/tmpkE0Y7wfieldobj_list_array_met-ws-720e07_6316.npy
reading took 0:00:03.731874
read to tmp-file /tmp/user/13098/tmpkE0Y7wfieldobj_list_array_met-ws-720e07_

2016-06-03 00:00:00
read to tmp-file /tmp/user/13098/tmpkE0Y7wfieldobj_list_array_met-ws-720e07_6316.npy
reading took 0:00:03.966152
read to tmp-file /tmp/user/13098/tmpkE0Y7wfieldobj_list_array_met-ws-720e07_6316.npy
reading took 0:00:03.917525
read to tmp-file /tmp/user/13098/tmpkE0Y7wfieldobj_list_array_met-ws-720e07_6316.npy
reading took 0:00:03.608140
read to tmp-file /tmp/user/13098/tmpkE0Y7wfieldobj_list_array_met-ws-720e07_6316.npy
reading took 0:00:03.722451
read to tmp-file /tmp/user/13098/tmpkE0Y7wfieldobj_list_array_met-ws-720e07_6316.npy
reading took 0:00:03.849366
read to tmp-file /tmp/user/13098/tmpkE0Y7wfieldobj_list_array_met-ws-720e07_6316.npy
reading took 0:00:03.977807
read to tmp-file /tmp/user/13098/tmpkE0Y7wfieldobj_list_array_met-ws-720e07_6316.npy
reading took 0:00:03.955100
read to tmp-file /tmp/user/13098/tmpkE0Y7wfieldobj_list_array_met-ws-720e07_6316.npy
reading took 0:00:03.898631
read to tmp-file /tmp/user/13098/tmpkE0Y7wfieldobj_list_array_met-ws-720e07_

2016-06-04 12:00:00
read to tmp-file /tmp/user/13098/tmpkE0Y7wfieldobj_list_array_met-ws-720e07_6316.npy
reading took 0:00:03.928299
read to tmp-file /tmp/user/13098/tmpkE0Y7wfieldobj_list_array_met-ws-720e07_6316.npy
reading took 0:00:03.729165
read to tmp-file /tmp/user/13098/tmpkE0Y7wfieldobj_list_array_met-ws-720e07_6316.npy
reading took 0:00:04.095980
read to tmp-file /tmp/user/13098/tmpkE0Y7wfieldobj_list_array_met-ws-720e07_6316.npy
reading took 0:00:03.930156
read to tmp-file /tmp/user/13098/tmpkE0Y7wfieldobj_list_array_met-ws-720e07_6316.npy
reading took 0:00:03.854550
read to tmp-file /tmp/user/13098/tmpkE0Y7wfieldobj_list_array_met-ws-720e07_6316.npy
reading took 0:00:03.918709
read to tmp-file /tmp/user/13098/tmpkE0Y7wfieldobj_list_array_met-ws-720e07_6316.npy
reading took 0:00:03.803142
read to tmp-file /tmp/user/13098/tmpkE0Y7wfieldobj_list_array_met-ws-720e07_6316.npy
reading took 0:00:04.005942
read to tmp-file /tmp/user/13098/tmpkE0Y7wfieldobj_list_array_met-ws-720e07_

2016-06-06 00:00:00
read to tmp-file /tmp/user/13098/tmpkE0Y7wfieldobj_list_array_met-ws-720e07_6316.npy
reading took 0:00:03.997738
read to tmp-file /tmp/user/13098/tmpkE0Y7wfieldobj_list_array_met-ws-720e07_6316.npy
reading took 0:00:04.476197
read to tmp-file /tmp/user/13098/tmpkE0Y7wfieldobj_list_array_met-ws-720e07_6316.npy
reading took 0:00:04.563911
read to tmp-file /tmp/user/13098/tmpkE0Y7wfieldobj_list_array_met-ws-720e07_6316.npy
reading took 0:00:04.118084
read to tmp-file /tmp/user/13098/tmpkE0Y7wfieldobj_list_array_met-ws-720e07_6316.npy
reading took 0:00:04.125948
read to tmp-file /tmp/user/13098/tmpkE0Y7wfieldobj_list_array_met-ws-720e07_6316.npy
reading took 0:00:04.397300
read to tmp-file /tmp/user/13098/tmpkE0Y7wfieldobj_list_array_met-ws-720e07_6316.npy
reading took 0:00:04.045207
read to tmp-file /tmp/user/13098/tmpkE0Y7wfieldobj_list_array_met-ws-720e07_6316.npy
reading took 0:00:04.342996
read to tmp-file /tmp/user/13098/tmpkE0Y7wfieldobj_list_array_met-ws-720e07_

2016-06-07 12:00:00
read to tmp-file /tmp/user/13098/tmpkE0Y7wfieldobj_list_array_met-ws-720e07_6316.npy
reading took 0:00:03.786195
read to tmp-file /tmp/user/13098/tmpkE0Y7wfieldobj_list_array_met-ws-720e07_6316.npy
reading took 0:00:03.902080
read to tmp-file /tmp/user/13098/tmpkE0Y7wfieldobj_list_array_met-ws-720e07_6316.npy
reading took 0:00:04.155629
read to tmp-file /tmp/user/13098/tmpkE0Y7wfieldobj_list_array_met-ws-720e07_6316.npy
reading took 0:00:03.962080
read to tmp-file /tmp/user/13098/tmpkE0Y7wfieldobj_list_array_met-ws-720e07_6316.npy
reading took 0:00:03.949962
read to tmp-file /tmp/user/13098/tmpkE0Y7wfieldobj_list_array_met-ws-720e07_6316.npy
reading took 0:00:04.221924
read to tmp-file /tmp/user/13098/tmpkE0Y7wfieldobj_list_array_met-ws-720e07_6316.npy
reading took 0:00:04.090533
read to tmp-file /tmp/user/13098/tmpkE0Y7wfieldobj_list_array_met-ws-720e07_6316.npy
reading took 0:00:03.973882
read to tmp-file /tmp/user/13098/tmpkE0Y7wfieldobj_list_array_met-ws-720e07_